In [2]:
import numpy as np

In [ ]:
class Softmax_Classifier:
    def __init__(self,epochs=1000,learning_rate=0.1):
        self.w=None
        self.b=None
        self.epochs=epochs
        self.learning_rate=learning_rate
        
        